# 07 Multiple Regression, Dummy-Codierung & Panelmodelle

In diesem Notebook erweitern wir die bisherige Regressionsanalyse um Themen aus Vorlesung 11 – Multiple Regression & Panelmodelle. Gemäß den Lernzielen der Vorlesung sollen wir
- multiple lineare Modelle formulieren und interpretieren,
- Dummy-Variablen korrekt erstellen und deuten,
- Multikollinearität diagnostizieren (mittels `Variance Inflation Factor`), und
- einen ersten Einblick in Panelmodelle gewinnen.

Wir nutzen weiterhin den OECD-Datensatz aus Notebook 05. Für die Regressionsmodelle benötigen wir eine Beobachtung pro Land (Pseudoreplikation vermeiden), weshalb wir den **Snapshot-Datensatz** `oecd_snapshot_latest.csv` verwenden. Für das Panelmodell (Fixed Effects) greifen wir auf den **Zeitreihendatensatz** `oecd_full_time_series.csv` zurück.

*Hinweis:* Die folgenden Codezellen laden die Daten aus dem Repository und führen die Berechnungen aus. Falls Sie dieses Notebook außerhalb des Projekts ausführen, passen Sie die Dateipfade entsprechend an.

## 1. Daten laden und vorbereiten

Wir importieren zunächst die benötigten Pakete. Anschließend laden wir den Snapshot-Datensatz und bereiten ihn analog zu Notebook 05 vor. Dabei fokussieren wir uns auf die Variablen *FeelingSafe* (abhängige Variable), *Homicides*, *SocialSupport* und *LifeSat*. Weitere potenzielle Einflussfaktoren können je nach Analyse ergänzt werden.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from linearmodels.panel import PanelOLS
from pathlib import Path

# Ein schöneres Plot-Design verwenden
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Dateipfade definieren
base_dir = Path.cwd()
snapshot_file = 'oecd_snapshot_latest.csv'
timeseries_file = 'oecd_full_time_series.csv'

# Versuchen, die Dateien im lokalen Projekt zu finden
candidate_dirs = [base_dir/'data', base_dir.parent/'data', Path('data')]
snapshot_path = None
timeseries_path = None
for d in candidate_dirs:
    if snapshot_path is None and (d / snapshot_file).exists():
        snapshot_path = d / snapshot_file
    if timeseries_path is None and (d / timeseries_file).exists():
        timeseries_path = d / timeseries_file

if snapshot_path is None:
    raise FileNotFoundError(f'Konnte {snapshot_file} nicht finden. Bitte Pfad anpassen.')
if timeseries_path is None:
    raise FileNotFoundError(f'Konnte {timeseries_file} nicht finden. Bitte Pfad anpassen.')

# Daten laden
df_snap = pd.read_csv(snapshot_path)
df_ts = pd.read_csv(timeseries_path)

# Daten kurz inspizieren
display(df_snap.head())
display(df_ts.head())


### 1.1 Snapshot-Datensatz aufbereiten

Wir filtern auf die relevanten Kennzahlen und erzeugen anschließend ein breites Format mit je einer Zeile pro Land. Damit erhalten wir die Matrix der Prädiktoren. Um starke Schiefe zu korrigieren, bilden wir den Logarithmus der Mordrate.

In [ ]:

# Variablen von Interesse
measures = ['Feeling safe at night', 'Homicides', 'Social support', 'Life satisfaction']

df_filtered = df_snap[df_snap['measure'].isin(measures)].copy()
# Numerische Werte erzwingen
df_filtered['value'] = pd.to_numeric(df_filtered['value'], errors='coerce')
# Pivotieren: eine Zeile pro Land, Spalten nach Kennzahl
df_pivot = df_filtered.pivot_table(index='reference_area', columns='measure', values='value', aggfunc='mean')
# Zeilen mit fehlenden Werten entfernen
df_pivot = df_pivot.dropna()
# Spaltennamen vereinfachen
df_pivot = df_pivot.rename(columns={
    'Feeling safe at night': 'FeelingSafe',
    'Homicides': 'Homicides',
    'Social support': 'SocialSupport',
    'Life satisfaction': 'LifeSat'
})

# Logarithmus der Mordrate bilden (kleine Konstante gegen log(0))
df_pivot['Log_Homicides'] = np.log(df_pivot['Homicides'] + 1e-6)

print(f'Anzahl Länder im Datensatz: {len(df_pivot)}')
display(df_pivot.head())


## 2. Multiple Regression

Die Multiple Regression ermöglicht es, den isolierten Effekt jeder erklärenden Variable zu schätzen, während alle anderen Variablen konstant gehalten werden (ceteris-paribus-Interpretation). Wie in der Vorlesung erläutert, trennt sie Einflussfaktoren voneinander und erlaubt realistischere Modelle als die einfache Regression.

In [ ]:

# Regressionsdesign
Y = df_pivot['FeelingSafe']
X = df_pivot[['Log_Homicides', 'SocialSupport', 'LifeSat']]
# Konstante hinzufügen
X = sm.add_constant(X)

# Modell schätzen
model_multi = sm.OLS(Y, X).fit()
print(model_multi.summary())


#### Interpretation

Die obige Tabelle zeigt Koeffizienten, Standardfehler, t-Werte und p-Werte des multiplen Modells. Wichtig ist die Interpretation der Steigungsparameter:

- Der Koeffizient für `Log_Homicides` misst, wie stark das Sicherheitsgefühl sinkt, wenn die logarithmierte Mordrate um eine Einheit steigt, **bei konstanter SocialSupport und LifeSat**.
- `SocialSupport` zeigt, wie stark die Unterstützung das Sicherheitsgefühl erhöht, wenn Mordrate und Lebenszufriedenheit konstant gehalten werden.
- `LifeSat` spiegelt den Zusammenhang zwischen allgemeiner Lebenszufriedenheit und Sicherheitsgefühl wider.

Ein signifikantes F-Statistik (p < 0,05) und ein relativ hoher Adjusted R² deuten darauf hin, dass das Modell einen substantiellen Anteil der Varianz erklärt.

## 3. Dummy-Variablen

Um den Umgang mit kategorialen Einflussgrößen zu demonstrieren, erstellen wir eine Dummy-Variable. Da der Snapshot-Datensatz keine offensichtliche Kategorisierung wie Geschlecht oder Bildungsgrad enthält (nach dem Pivotieren liegt eine Zeile pro Land vor), klassifizieren wir Länder anhand ihrer sozialen Unterstützung: Länder mit überdurchschnittlicher SocialSupport erhalten die Dummy-Variable `HighSocial` = 1, alle anderen 0.

Dummy-Variablen (kategoriale Variablen korrekt codieren)

In VL11 geht es bei Dummy-Codierung vor allem darum, **kategoriale Merkmale** (z. B. Länder, Regionen, Kontinente) in ein Regressionsmodell zu integrieren.

Wichtig:
- Für eine kategoriale Variable mit `k` Kategorien erzeugen wir **k–1 Dummies**.
- Eine Kategorie wird als **Referenzkategorie** ausgelassen.
- Die Dummy-Koeffizienten sind dann **Niveauunterschiede zur Referenz**, *ceteris paribus* (unter Kontrolle der anderen Prädiktoren).

Da unser Snapshot-Datensatz sehr viele Länder enthält, wählen wir für eine gut interpretierbare Demo eine kleine Länder-Auswahl.


In [ ]:
# --- Demo-Auswahl: wenige Länder für interpretierbare Dummy-Koeffizienten ---
# (Du kannst die Liste gern manuell anpassen.)
demo_countries = ['Switzerland', 'Germany', 'France', 'Italy', 'United States', 'Japan']
available = [c for c in demo_countries if c in df_pivot.index]

if len(available) < 3:
    # Fallback: nimm die ersten 6 Länder aus dem Datensatz
    available = list(df_pivot.index[:6])

df_demo = df_pivot.loc[available].copy()
df_demo['reference_area'] = df_demo.index  # Land als Spalte

display(df_demo)
print("Verwendete Länder:", available)


In [ ]:
# --- Demo-Auswahl: wenige Länder für interpretierbare Dummy-Koeffizienten ---
# (Du kannst die Liste gern manuell anpassen.)
demo_countries = ['Switzerland', 'Germany', 'France', 'Italy', 'United States', 'Japan']
available = [c for c in demo_countries if c in df_pivot.index]

if len(available) < 3:
    # Fallback: nimm die ersten 6 Länder aus dem Datensatz
    available = list(df_pivot.index[:6])

df_demo = df_pivot.loc[available].copy()
df_demo['reference_area'] = df_demo.index  # Land als Spalte

display(df_demo)
print("Verwendete Länder:", available)


#### Interpretation (Dummy-Variablen)

- Die ausgelassene Kategorie (Referenz) ist **das erste Land der Liste**.  
- Jeder Dummy-Koeffizient (z. B. `country_Germany`) ist der **durchschnittliche Niveauunterschied** in `LifeSat` zwischen Deutschland und der Referenz **unter Kontrolle von** `Log_Homicides` und `SocialSupport`.
- Wichtig: Dummy-Koeffizienten sind **keine Kausalität**, sondern kontrollierte Niveauvergleiche.


## 4. Multikollinearität diagnostizieren

Mehrere erklärende Variablen können untereinander stark korreliert sein, was zu instabilen Koeffizientenschätzungen führt. In der Vorlesung wurde dazu der **Variance Inflation Factor (VIF)** vorgestellt. Für jede Variable berechnen wir den VIF – Werte über 5–10 gelten als Hinweis auf problematische Multikollinearität.

In [ ]:

# VIF-Berechnung für das multiple Modell
vif_data = pd.DataFrame()
vif_data['Variable'] = ['Log_Homicides', 'SocialSupport', 'LifeSat']
vif_data['VIF'] = [variance_inflation_factor(df_pivot[['Log_Homicides', 'SocialSupport', 'LifeSat']].values, i)
                    for i in range(3)]
vif_data


#### Interpretation

Wenn die VIF-Werte unter 5 liegen, besteht keine ernsthafte Multikollinearität. Höhere Werte legen nahe, dass einzelne Prädiktoren stark mit anderen korrelieren. In diesem Fall sollte geprüft werden, ob alle Variablen benötigt werden oder ob bestimmte Variablen kombiniert oder entfernt werden sollten.

## 5. Konfundierung (Confounding)

Konfundierung beschreibt das Phänomen, dass der Effekt einer Variable auf die Zielgröße durch andere Variablen verfälscht wird. Im Beispiel aus Notebook 05 führte die Berücksichtigung von `SocialSupport` dazu, dass der Effekt der Mordrate geringer wurde. Wir demonstrieren diesen Effekt, indem wir das Modell mit und ohne SocialSupport schätzen und die Änderung des Koeffizienten für `Log_Homicides` vergleichen.

In [ ]:

# Modell ohne SocialSupport
X_simple = sm.add_constant(df_pivot[['Log_Homicides']])
model_simple = sm.OLS(Y, X_simple).fit()

# Modell mit SocialSupport
X_with_support = sm.add_constant(df_pivot[['Log_Homicides', 'SocialSupport']])
model_with_support = sm.OLS(Y, X_with_support).fit()

print('Koeffizient Log_Homicides ohne SocialSupport:', model_simple.params['Log_Homicides'])
print('Koeffizient Log_Homicides mit SocialSupport:', model_with_support.params['Log_Homicides'])


#### Interpretation

Wenn der Koeffizient für `Log_Homicides` im Modell ohne SocialSupport deutlich größer ist als im Modell mit SocialSupport, dann wirkt `SocialSupport` als **Konfundierer**: Die soziale Unterstützung hängt negativ mit der Mordrate zusammen und positiv mit dem Sicherheitsgefühl. Ohne Kontrolle für SocialSupport überschätzt das Modell den (negativen) Zusammenhang zwischen Mordrate und Sicherheitsgefühl.

## 6. Panelmodell (Fixed Effects)

Während Multiple Regression (Querschnitt) Zusammenhänge in einem Zeitpunkt analysiert, erlaubt Panelanalyse zusätzlich die Trennung von:

- **Between-Variation**: Unterschiede zwischen Ländern (dauerhafte Niveauniveaus)
- **Within-Variation**: Veränderungen innerhalb eines Landes über die Zeit

Wir vergleichen drei Modelle:
1) **Pooled OLS** (ignoriert Panelstruktur)
2) **Entity Fixed Effects** (kontrolliert zeitinvariante Länderunterschiede)
3) **Two-Way Fixed Effects** (zusätzlich Zeit-Fixeffekte pro Jahr)

Damit decken wir die zentralen Inhalte aus VL11 zu Paneldaten und Fixed Effects ab.


In [ ]:
from linearmodels.panel import PanelOLS
import numpy as np
import pandas as pd

# --- Relevante Kennzahlen für das Panel ---
measures_panel = ['Feeling safe at night', 'Homicides', 'Social support']
df_ts_filt = df_ts[df_ts['measure'].isin(measures_panel)].copy()

# numeric
df_ts_filt['value'] = pd.to_numeric(df_ts_filt['value'], errors='coerce')

# Pivot: eine Zeile pro Land/Jahr
df_ts_panel = df_ts_filt.pivot_table(
    index=['reference_area', 'year'],
    columns='measure',
    values='value',
    aggfunc='mean'
)

# Umbenennen
df_ts_panel = df_ts_panel.rename(columns={
    'Feeling safe at night': 'FeelingSafe',
    'Homicides': 'Homicides',
    'Social support': 'SocialSupport'
})

# Nur droppen, was fürs Modell nötig ist
df_ts_panel = df_ts_panel.dropna(subset=['FeelingSafe', 'Homicides', 'SocialSupport'])

# Log-Transformation (stabilisiert Skalen, reduziert Ausreißer-Einfluss)
df_ts_panel['Log_Homicides'] = np.log(df_ts_panel['Homicides'] + 1e-6)

# Panel-Index setzen (Entity = Land, Time = Jahr)
panel_df = df_ts_panel.reset_index().set_index(['reference_area', 'year']).sort_index()

display(panel_df.head())
print("Panel shape:", panel_df.shape)
print("Anzahl Länder:", panel_df.index.get_level_values(0).nunique())
print("Zeitraum:", panel_df.index.get_level_values(1).min(), "-", panel_df.index.get_level_values(1).max())


Das vorliegende Panel ist **unbalanciert**, da nicht für jedes Land in jedem Jahr
Beobachtungen vorliegen. Die verwendeten Fixed-Effects-Modelle können auch in
diesem Fall konsistent geschätzt werden.


### Modellschätzung

Wir nutzen **cluster-robuste Standardfehler auf Länderebene**, da Beobachtungen innerhalb eines Landes über die Zeit korreliert sein können.

- **Pooled OLS**: keine Fixed Effects → misst gemischte Between+Within Effekte
- **Entity FE**: kontrolliert alle zeitinvarianten Länderfaktoren (z. B. Kultur, Institutionen)
- **Two-Way FE**: zusätzlich Year-FE → kontrolliert globale Trends/Schocks (z. B. allgemeine Entwicklungen, Krisen)


In [ ]:
# 1) Pooled OLS (ohne FE)
pooled = PanelOLS.from_formula(
    'FeelingSafe ~ 1 + Log_Homicides + SocialSupport',
    data=panel_df
).fit(cov_type="clustered", cluster_entity=True)

# 2) Entity Fixed Effects (Länder-FE)
fe_entity = PanelOLS.from_formula(
    'FeelingSafe ~ 1 + Log_Homicides + SocialSupport + EntityEffects',
    data=panel_df
).fit(cov_type="clustered", cluster_entity=True)

# 3) Two-Way Fixed Effects (Länder + Zeit)
fe_tw = PanelOLS.from_formula(
    'FeelingSafe ~ 1 + Log_Homicides + SocialSupport + EntityEffects + TimeEffects',
    data=panel_df
).fit(cov_type="clustered", cluster_entity=True)

print("=== Pooled OLS ===")
print(pooled.summary)

print("\n=== Entity Fixed Effects ===")
print(fe_entity.summary)

print("\n=== Two-Way Fixed Effects ===")
print(fe_tw.summary)


In [ ]:
comparison = pd.DataFrame({
    "Pooled_coef": pooled.params,
    "Pooled_se": pooled.std_errors,
    "Pooled_p": pooled.pvalues,

    "FE_Entity_coef": fe_entity.params,
    "FE_Entity_se": fe_entity.std_errors,
    "FE_Entity_p": fe_entity.pvalues,

    "FE_TwoWay_coef": fe_tw.params,
    "FE_TwoWay_se": fe_tw.std_errors,
    "FE_TwoWay_p": fe_tw.pvalues
})

display(comparison)


### Interpretation (VL11)

**1) Pooled OLS**
- Interpretiert Koeffizienten als Zusammenhang über alle Beobachtungen hinweg.
- Problem: Unterschiede zwischen Ländern (z. B. stabile Institutionen, Sicherheitskultur) können die erklärenden Variablen und `FeelingSafe` gleichzeitig beeinflussen.
- Dadurch können Koeffizienten **verzerrt** sein (Omitted Variable Bias durch unbeobachtete, zeitinvariante Faktoren).

**2) Entity Fixed Effects (Länder-FE)**
- Kontrolliert automatisch alle **zeitinvarianten** Unterschiede zwischen Ländern.
- Interpretation der Koeffizienten ist eine **Within-Aussage**:
  *Wenn sich innerhalb eines Landes `Log_Homicides` oder `SocialSupport` über die Zeit verändert, wie verändert sich dann `FeelingSafe`?*

**3) Two-Way Fixed Effects (Länder + Jahr)**
- Zusätzlich zu Länder-FE werden Jahres-FE hinzugefügt.
- Das kontrolliert globale Zeittrends/Schocks, die alle Länder betreffen könnten.
- Wenn sich Koeffizienten zwischen Entity-FE und Two-Way-FE deutlich ändern, spricht das dafür, dass ein Teil des Zusammenhangs vorher durch **Zeittrends** erklärt wurde.

**Praktischer Vergleich**
- Wenn Pooled stark von FE abweicht, ist das ein Hinweis, dass **Between-Unterschiede** eine große Rolle spielen.
- FE-Modelle sind oft näher an einer „ceteris paribus“-Interpretation innerhalb einer Einheit (Land), bleiben aber weiterhin **assoziativ** (nicht automatisch kausal).


### Fixed Effects light: Demeaning (Within-Transformation)

Eine alternative Sicht auf Entity Fixed Effects ist Demeaning:
Für jedes Land ziehen wir den Länder-Mittelwert ab (Within-Transformation).  
Danach wird eine OLS-Regression auf den demeaned Variablen geschätzt.


In [ ]:
import statsmodels.api as sm

# Within-Transformation pro Land (Entity)
vars_fe = ['FeelingSafe', 'Log_Homicides', 'SocialSupport']
df_dm = panel_df[vars_fe].groupby(level=0).transform(lambda g: g - g.mean())

y_dm = df_dm['FeelingSafe']
X_dm = df_dm[['Log_Homicides', 'SocialSupport']]

# keine Konstante (nach Demeaning ~0)
demean_model = sm.OLS(y_dm, X_dm).fit()
print(demean_model.summary())


## 7. Fazit

In diesem Notebook wurden zentrale Konzepte aus **Vorlesung 11** systematisch auf den Datensatz angewendet und erweitert:

- **Multiple Regression** ermöglicht die Schätzung von **partiellen Effekten**, indem mehrere Einflussfaktoren gleichzeitig kontrolliert werden.
- **Dummy-Codierung** wurde explizit für **kategoriale Variablen** (k–1 Dummies mit Referenzkategorie) umgesetzt, wodurch Niveauunterschiede zwischen Gruppen unter Kontrolle weiterer Prädiktoren interpretierbar werden.
- **Multikollinearität** wurde mithilfe des **Variance Inflation Factor (VIF)** diagnostiziert, um potenziell instabile Koeffizientenschätzungen zu identifizieren.
- **Konfundierung** wurde aufgezeigt, indem sich Koeffizienten beim Hinzufügen weiterer erklärender Variablen veränderten, was die Bedeutung kontrollierter Modellierung unterstreicht.
- **Panelmodelle** wurden in Form eines **Modellvergleichs zwischen Pooled OLS, Entity Fixed Effects und Two-Way Fixed Effects** geschätzt.
  Dadurch konnte zwischen **Between-Effekten** (Unterschiede zwischen Ländern) und **Within-Effekten** (Veränderungen innerhalb eines Landes über die Zeit) unterschieden werden.
  Fixed Effects erlauben es, unbeobachtete zeitinvariante Unterschiede zu kontrollieren und führen damit zu robusteren Schätzungen in längsschnittlichen Daten.

Insgesamt fügt sich dieses Notebook in die in den Vorlesungen vermittelte Lernlogik ein und zeigt, wie multiple Regression, Dummy-Codierung und Panelmethoden kombiniert werden können, um komplexe Zusammenhänge in ökonomischen und sozialen Daten fundiert zu analysieren.
